In [2]:
from PIL import Image
import torchvision.transforms as transforms
from diffusers import StableDiffusionInpaintPipeline
import torch
from pathlib import Path
import os

In [27]:
ref_image_path = Path('./training_data/phil/phil_head.png')
ref_image = Image.open(ref_image_path)

In [22]:

train_transform = transforms.Compose([
    transforms.Resize(512),
    transforms.CenterCrop(512),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.Pad(30),
    transforms.Resize(512)
])

transformed_image = train_transform(ref_image)


In [18]:
def compute_transparency_mask(image):
    """Compute transparency mask for an image.
    Args:
        image (PIL.Image): Input image.
    Returns:
        PIL.Image: Inverted Transparency mask.
    """
    if image.mode != 'RGBA':
        image = image.convert('RGBA')
    alpha = image.split()[3]
    mask = Image.eval(alpha, lambda a: 255 if a <=128 else 0)
    return mask

In [28]:
def outpaint(pipe_inpaint, image, mask):
    prompt = "a photo of a person, highly detailed, soft natural lighting, photo realism, professional portrait, ultra-detailed, 4k resolution, wallpaper, hd wallpaper"
    negative_prompt = "magzine, frame, tiled, repeated, multiple people, multiple faces, group of people, split frame, multiple panel, split image, watermark, boarder, diptych, triptych"
    image = pipe_inpaint(prompt=prompt, 
                 image=image, 
                 mask_image=mask,
                 num_inference_steps=50).images[0]
    return image

    

In [ ]:
nb_images = 10

save_folder = './training_data/' + ref_image_path.stem + '_augmented'
save_folder = 
if not os.path.exists(save_folder):
    os.makedirs(save_folder)


pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
).to("cuda")

for i in range(nb_images):
    transformed_image = train_transform(ref_image)
    mask = compute_transparency_mask(transformed_image)
    outpainted_image = outpaint(pipe_inpaint, transformed_image, mask)
    outpainted_image.save(save_folder + f'/{i}.png')
    print(f'Image {i} done')
    outpainted_image.show()